<a href="https://colab.research.google.com/github/RazerRaymond/Pricing-Simulations/blob/main/greeks_3_different_approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import math
import scipy.stats as st  

In [ ]:
# Data for Stock
S0 = 100  # spot price
r = 0.05  # risk-free interest rate
q = 0 # div rate
sig = 0.38  # volatility of stock price
T = 0.5  # maturity date

delS = 0.1
delSig = 0.01

nN = 250000

In [ ]:
# Option data
t1 = 1/6.0
lamb = 1.1

### CRN

In [ ]:
np.random.seed(43452456)
z = np.random.randn(nN)
z1 = np.random.randn(nN)

In [ ]:
# update_k1 is a multiplier that transform S0 to St1
update_t1 = np.exp(((r-q) - (sig**2) / 2) * t1 + sig * np.sqrt(t1) * z) 
K = lamb * S0 * update_t1 # Strike price array
St1 = S0 * update_t1 # St1 array

In [ ]:
tmp1U = (S0 + delS)*update_t1*np.exp((r-q-sig**2/2)*(T-t1))
tmp1 = St1 * np.exp((r-q-sig**2/2)*(T-t1))
tmp1D = (S0 - delS)*update_t1*np.exp((r-q-sig**2/2)*(T-t1))

tmp2 = sig*np.sqrt((T-t1))


s = tmp1*np.exp(tmp2*z1)
tmp = np.exp(-r*T)*np.maximum(s-K,0)

s = tmp1U*np.exp(tmp2*z1)
tmpU = np.exp(-r*T)*np.maximum(s-K,0)

s = tmp1D*np.exp(tmp2*z1)
tmpD = np.exp(-r*T)*np.maximum(s-K,0)

delC_CRN1 = np.mean((tmpU - tmpD))/(2*delS)

gam_CRN  = np.mean((tmpU - 2*tmp + tmpD))/(delS**2)

In [ ]:
ump = np.exp(((r-q) - ((sig + delSig)**2) / 2) * t1 + (sig + delSig) * np.sqrt(t1) * z)
dmp = np.exp(((r-q) - ((sig - delSig)**2) / 2) * t1 + (sig - delSig) * np.sqrt(t1) * z)

KU = S0 * ump * lamb
KD = S0 * dmp * lamb
St1u = S0 * ump
St1d = S0 * dmp

In [ ]:
tmp1U  =  St1u * np.exp((r - q - (sig + delSig)**2/2)*(T-t1))
tmp1D  =  St1d * np.exp((r - q - (sig - delSig)**2/2)*(T-t1))

tmp2U = (sig + delSig) * np.sqrt((T-t1))
tmp2D = (sig - delSig) * np.sqrt((T-t1))


s = tmp1U * np.exp(tmp2U*z1)
tmpU = np.exp(-r*T) * np.maximum(s-KU,0)

s = tmp1D * np.exp(tmp2D*z1)
tmpD = np.exp(-r*T) * np.maximum(s-KD,0)

vega_CRN = np.mean((tmpU - tmpD)/(2*delSig))

### Pathwise

In [ ]:
z = np.random.randn(nN)
z1 = np.random.randn(nN)

In [ ]:
update_t1 = np.exp(((r-q) - (sig**2) / 2) * t1 + sig * np.sqrt(t1) * z)
K = lamb * S0 * update_t1
St1 = S0 * update_t1
s = St1 * np.exp(((r-q) - (sig**2) / 2) * (T-t1) + sig * np.sqrt((T-t1)) * z1)

der1 = np.exp((r - q - sig**2/2)*t1) * np.exp(sig*np.sqrt(t1)*z)
der2 = np.exp((r - q - sig**2/2)*(T-t1)) * np.exp(sig*np.sqrt((T-t1))*z1)
der3 =  np.exp(-r * T) * (s>K)

dhds0 = der1*der2*der3
delC_pathwise = np.mean(dhds0)
gam_pathwise = np.nan

In [ ]:
dhdsig = np.exp(-r*T) * (s>K) * (s/sig) * (np.log(s/S0) - (r - q+sig**2/2)*T)
vega_pathwise = np.mean(dhdsig)

### Likelihood

In [ ]:
z = np.random.randn(nN)
z1 = np.random.randn(nN)

In [ ]:
update_t1 = np.exp(((r-q) - (sig**2) / 2) * t1 + sig * np.sqrt(t1) * z)
K = lamb * S0 * update_t1
St1 = S0 * update_t1
s = St1 * np.exp(((r-q) - (sig**2) / 2) * (T-t1) + sig * np.sqrt((T-t1)) * z1)

In [ ]:
scoreH_delta = np.exp(-r*T)*np.maximum(s - K,0)*z1/(S0*sig*np.sqrt(T-t1))
delC_likelihood = np.mean(scoreH_delta)

scoreH_gamma = (z1**2 - 1)/(S0**2*sig**2*(T-t1)) - z1/(S0**2*sig*np.sqrt(T-t1))
scoreH_gamma *= np.exp(-r*T)*np.maximum(s - K,0)
gam_likelihood = np.mean(scoreH_gamma)

In [ ]:
scoreH = np.exp(-r*T)*np.maximum(s - K,0)*((z1**2-1)/sig - z1*np.sqrt(T-t1))
vega_likelihood = np.mean(scoreH)

In [ ]:
print('==================')
print('\tDelta')
print('==================')
print('W/ CRN: %f' % delC_CRN1)
print('Pathwise estimator: %f' % delC_pathwise)
print('Likelihood estimator: %f' % delC_likelihood)
print('==================')
print('\tGamma')
print('==================')
print('W/ CRN: %f' % gam_CRN)
print('Pathwise estimator: %f' % gam_pathwise)
print('Likelihood estimator: %f' % gam_likelihood)
print('==================')
print('\tVega')
print('==================')
print('W/ CRN: %f' % vega_CRN)
print('Pathwise estimator: %f' % vega_pathwise)
print('Likelihood estimator: %f' % vega_likelihood)

	Delta
W/ CRN: 0.401898
Pathwise estimator: 0.404425
Likelihood estimator: 0.404279
	Gamma
W/ CRN: 0.017204
Pathwise estimator: nan
Likelihood estimator: 0.017736
	Vega
W/ CRN: 22.287628
Pathwise estimator: 22.525262
Likelihood estimator: 22.466003
